In [1]:
!pip install -q -U autogluon.tabular

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.0 requires aiohttp<4.0.0,>=3.9.2, which is not installed.
s3fs 2024.3.1 requires aiohttp!=4.0.0a0,!=4.0.0a1, which is not installed.
aiobotocore 2.13.0 requires botocore<1.34.107,>=1.34.70, but you have botocore 1.29.165 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


# Imports and Configs

In [2]:
from autogluon.tabular import TabularPredictor, TabularDataset
import pandas as pd
import warnings
import shutil
import pickle
import glob

warnings.filterwarnings('ignore')

In [3]:
TIME_LIMIT = 3600 * 8
SEED = 27
TARGET = 'Target'

# Loading and Preprocessing the Datasets

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv', index_col='id')

train.shape, test.shape

((76518, 37), (51012, 36))

In [5]:
original = pd.read_csv('/kaggle/input/predict-students-dropout-and-academic-success/data.csv')
train = pd.concat([train, original], axis=0, ignore_index=True)

train.shape

(80942, 37)

In [6]:
train = TabularDataset(train)
test = TabularDataset(test)

# Training

In [7]:
predictor = TabularPredictor(
    label=TARGET,
    eval_metric='accuracy',
    problem_type='multiclass',
    verbosity=2
).fit(
    train_data=train,
    time_limit=TIME_LIMIT,
    presets='best_quality',
    excluded_model_types = ['KNN', 'NN', 'FASTAI', 'NN_TORCH', 'RF'],
    save_space=False,
    hyperparameter_tune_kwargs={  
        'num_trials': 50,
        'scheduler' : 'local',
        'searcher'  : 'auto'
    },
    feature_prune_kwargs={'force_prune': True}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240601_154308"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 28800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240601_154308/ds_sub_fit/sub_fit_ho.
2024-06-01 15:43:09,111	INFO util.py:124 -- Outdated packages:
  ipywidgets==

  0%|          | 0/50 [00:00<?, ?it/s]

Will use sequential fold fitting strategy because import of ray failed. Reason: ray==2.9.0 detected. 2.10.0 <= ray < 2.11.0 is required. You can use pip to install certain version of ray `pip install ray==2.10.0` 
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 209. Best iteration is:
	[193]	valid_set's multi_error: 0.174561
	Ran out of time, early stopping on iteration 211. Best iteration is:
	[207]	valid_set's multi_error: 0.17167
	Ran out of time, early stopping on iteration 220. Best iteration is:
	[215]	valid_set's multi_error: 0.17734
	Ran out of time, early stopping on iteration 216. Best iteration is:
	[216]	valid_set's multi_error: 0.170892
	Ran out of time, early stopping on iteration 276. Best iteration is:
	[237]	valid_set's multi_error: 0.171578
	Ran out of time, early stopping on iteration 305. Best iteration is:
	[271]	valid_set's multi_error: 0.171133
	Ran out of time, early stopping o

  0%|          | 0/50 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 229. Best iteration is:
	[216]	valid_set's multi_error: 0.17167
	Ran out of time, early stopping on iteration 245. Best iteration is:
	[235]	valid_set's multi_error: 0.174116
	Ran out of time, early stopping on iteration 283. Best iteration is:
	[266]	valid_set's multi_error: 0.168242
	Stopping HPO to satisfy time limit...
Fitted model: LightGBM_BAG_L1/T1 ...
	0.83	 = Validation score   (accuracy)
	59.46s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ... Tuning model for up to 78.52s of the 7064.79s of remaining time.


...

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 71. Best iteration is:
	[33]	valid_set's multi_error: 0.167227
	Ran out of time, early stopping on iteration 76. Best iteration is:
	[37]	valid_set's multi_error: 0.166831
	Ran out of time, early stopping on iteration 63. Best iteration is:
	[25]	valid_set's multi_error: 0.172465
	Ran out of time, early stopping on iteration 90. Best iteration is:
	[87]	valid_set's multi_error: 0.164855
	Stopping HPO to satisfy time limit...
Fitted model: LightGBM_BAG_L2/T1 ...
	0.8327	 = Validation score   (accuracy)
	195.09s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L2 ... Tuning model for up to 221.06s of the 13132.7s of remaining time.


  0%|          | 0/50 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 56.
	Ran out of time, early stopping on iteration 62.
	Ran out of time, early stopping on iteration 69.
	Ran out of time, early stopping on iteration 73.
	Ran out of time, early stopping on iteration 80.
	Ran out of time, early stopping on iteration 98.
	Stopping HPO to satisfy time limit...
Fitted model: CatBoost_BAG_L2/T1 ...
	0.8321	 = Validation score   (accuracy)
	206.83s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: ExtraTreesGini_BAG_L2 ... Tuning model for up to 221.06s of the 12924.42s of remaining time.
	No hyperparameter search space specified for ExtraTreesGini_BAG_L2. Skipping HPO. Will train one model based on the provided hyperparameters.
Fitted model: ExtraTreesGini_BAG_L2 ...
	0.8304	 = Validation score   (accuracy)
	48.93s	 = Training   runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: Ext

  0%|          | 0/50 [00:00<?, ?it/s]

	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Stopping HPO to satisfy time limit...
Fitted model: XGBoost_BAG_L2/T1 ...
	0.8329	 = Validation score   (accuracy)
	137.98s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L2 ... Training model for up to 221.06s of the 12687.53s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 52. Best iteration is:
	[50]	valid_set's multi_error: 0.160803
	Ran out of time, early stopping on iteration 53. Best iteration is:
	[53]	valid_set's multi_error: 0.167919
	Ran out of time, early stopping on iteration 50. Best iteration is:
	[50]	valid_set's multi_error: 0.168215
	Ran out of time, early stopping on iteration 36. Best iteration is:
	[36]	valid_set's multi_error: 0.170686
	Ran out of time, early stopping on iteration 58. Best iteration is:
	[36]	valid_set's multi_error: 0.17

In [8]:
oof_predictions = predictor.get_oof_pred_proba()
with open('autogluon_oof_preds.pkl', 'wb') as f:
    pickle.dump(oof_predictions, f)

# Inference

In [9]:
preds = predictor.predict_proba(test)

In [10]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv')
submission[TARGET] = preds.idxmax(axis=1).values
submission.to_csv('submission.csv', index=False)
submission.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled


In [11]:
shutil.rmtree("AutogluonModels")